In [44]:
import pandas as pd
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [2]:
# Reading the csv file and transforming the data using the code from EX1:
df = pd.read_csv('CVD_cleaned.csv')
# Choosing relevant columns:
df = df[['Sex', 'Alcohol_Consumption']]

# Changing categorical variable into binary values
df['Sex'] = df['Sex'].replace(['Male', 'Female'], [1, 2])
df

,Sex,Alcohol_Consumption
0,2,0.0
1,2,0.0
2,2,4.0
3,1,0.0
4,1,0.0
...,...,...
308849,1,4.0
308850,1,8.0
308851,2,4.0
308852,1,3.0


# Research question is:
###### Does alcohol consumption vary between men and women?
* X - Alcohol_Consumption
* Y - Sex (2 categories: 1 - male, 0 - female)

## Q 1

In [5]:
# Sampling 200 rows:
df_sample_200 = df.sample(n=200, random_state=42)

# Getting the indices of the already sampled rows
sampled_indices = df_sample_200.index

# Excluding the already sampled rows from the DataFrame
remaining_df = df.drop(sampled_indices)

# Sampling another 1000 rows from the remaining DataFrame
df_sample_1000 = remaining_df.sample(n=1000, random_state=42)

## Q 2

### Q 2.1

In [6]:
def eta(p):
    epsilon = 1e-10 # Small constant to prevent division by zero
    return np.log(p / (1 - p + epsilon))


In [7]:
# Setting tau to be the median of Alcohol_Consumption
tau = df['Alcohol_Consumption'].median()

In [8]:
# Estimating p1 and p2:
len_male = df_sample_200[(df_sample_200['Sex'] == 1)].shape[0]
len_female = 200 - len_male
p1 = df_sample_200[(df_sample_200['Sex'] == 1) & (df_sample_200['Alcohol_Consumption'] > tau)].shape[0] / len_male
p2 = df_sample_200[(df_sample_200['Sex'] == 2) & (df_sample_200['Alcohol_Consumption'] > tau)].shape[0] / len_female
print("p1:", p1)
print("p2:", p2)

p1: 0.5714285714285714
p2: 0.39215686274509803


In [9]:
# Estimating phi
phi = eta(p1) - eta(p2)
print("phi: ", phi)

phi:  0.725937003314119


In [10]:
# Confiedence Interval of phi using Bootstrap:
B = 700
phi_values = []
alpha = 0.05
for i in range(B):
    resampled_df = df_sample_200.sample(n=len(df_sample_200), replace=True)
    p1 = resampled_df[(resampled_df['Sex'] == 1) & (resampled_df['Alcohol_Consumption'] > tau)].shape[0] / len_male
    p2 = resampled_df[(resampled_df['Sex'] == 2) & (resampled_df['Alcohol_Consumption'] > tau)].shape[0] / len_female
    phi_values.append(eta(p1) - eta(p2))
    


In [43]:
# Compute confidence interval using percentile method
lower_percentile = (alpha / 2) * 100
upper_percentile = 100 - lower_percentile
lower_bound = np.percentile(phi_values, lower_percentile)
upper_bound = np.percentile(phi_values, upper_percentile)
print(f"Confidence Interval for Phi: [{lower_bound, upper_bound}]")
print(f"Legnth of CI: {upper_bound - lower_bound}")

Confidence Interval for Phi: [(-0.04595672094505621, 1.5340384755016025)]
Legnth of CI: 1.5799951964466588


__As we can see phi is in the confidence interval calculated in the percentile method.__

### Q 2.2
__We'll employ the formula introduced in the lecture, where:__

* S represents the count of males/females with alcohol consumption exceeding tau.
* n denotes the total number of men/women.

In [39]:
# Estimating phi:
s_male = df_sample_200[(df_sample_200['Sex'] == 1) & (df_sample_200['Alcohol_Consumption'] > tau)].shape[0]
s_female = df_sample_200[(df_sample_200['Sex'] == 2) & (df_sample_200['Alcohol_Consumption'] > tau)].shape[0]
p1_exp = s_male / (len_male + 2)
p2_exp = s_female / (len_female + 2)
p1_mle = s_male / len_male
p2_mle = s_female / len_female
p1_est = (2 / (len_male + 2) * p1_exp) + (len_male / (len_male + 2) * p1_mle)
p2_est = (2 / (len_female + 2) * p2_exp) + (len_female / (len_female + 2) * p2_mle)

print(f"phi: {eta(p1_est) - eta(p2_est)}")

phi: 0.7256121897100627


In [58]:
# Calculating Credible Interval
phi_list = []
for i in range(B):
    p1_boot = stats.beta.rvs(s_male + 1, len_male - s_male + 1, size=1)
    p2_boot = stats.beta.rvs(s_female + 1, len_female - s_female + 1, size=1)
    phi_list.append(eta(p1_boot) - eta(p2_boot))

lower_bound2 = np.percentile(phi_list, lower_percentile)
upper_bound2 = np.percentile(phi_list, upper_percentile)
print(f"Credible Interval for Phi: {lower_bound2, upper_bound2}")
print(f"Legnth of CI: {upper_bound2 - lower_bound2}")

Credible Interval for Phi: (0.15300626243949325, 1.2208620866335897)
Legnth of CI: 1.0678558241940965


### Q 2.3
__Jeffrey's Prior__
* Postertior: sqrt(1/p(1-p)) * p^s(1-p)^n-s ---> ~ Beta(s + 0.5, n - s + 0.5)

In [61]:
phi_estimators = []
for i in range(B):
    p1_boot = stats.beta.rvs(s_male + 0.5, len_male - s_male + 0.5, size=1)
    p2_boot = stats.beta.rvs(s_female + 0.5, len_female - s_female + 0.5, size=1) 
    phi_estimators.append(eta(p1_boot) - eta(p2_boot))
    
lower_bound3 = np.percentile(phi_estimators, lower_percentile)
upper_bound3 = np.percentile(phi_estimators, upper_percentile)
print(f"Credible Interval for Phi using Jeffrey's Prior: {lower_bound3, upper_bound3}")
print(f"Legnth of CI: {upper_bound3 - lower_bound3}")
print(f"Phi estimator: {np.mean(phi_estimators)}")

Credible Interval for Phi using Jeffrey's Prior: (0.16937325250747706, 1.2554661332189538)
Legnth of CI: 1.0860928807114767
Phi estimator: 0.7164346878723177


### Q 2.4

In [69]:
# Counting the number of males and females in df_sample_1000
len_male_1000 = df_sample_1000[df_sample_1000['Sex'] == 1].shape[0]
len_female_1000 = df_sample_1000[df_sample_1000['Sex'] == 2].shape[0]
s1_male_1000 = df_sample_1000[(df_sample_1000['Sex'] == 1) & (df_sample_1000['Alcohol_Consumption'] > tau)].shape[0]
s2_female_1000 = df_sample_1000[(df_sample_1000['Sex'] == 2) & (df_sample_1000['Alcohol_Consumption'] > tau)].shape[0]

print(f"p1 prior parameters are: {s1_male_1000 + 1, len_male_1000 - s1_male_1000 + 1 }")
print(f"p2 prior parameters are: {s2_female_1000 + 1, len_female_1000 - s2_female_1000 + 1 }\n")

# Using power laws and we get:
print(f"p1 posterior parameters are: {s1_male_1000 + 262, len_male_1000 - s1_male_1000 + 219}")
print(f"p2 posterior parameters are: {s2_female_1000 + 221, len_female_1000 - s2_female_1000 + 302}")

p1 prior parameters are: (262, 219)
p2 prior parameters are: (221, 302)

p1 posterior parameters are: (523, 437)
p2 posterior parameters are: (441, 603)


In [66]:
phi_vals = []
for i in range(B):
    p1_boot = stats.beta.rvs(s1_male_1000 + 262, len_male_1000 - s1_male_1000 + 219, size=1)
    p2_boot = stats.beta.rvs(s2_female_1000 + 221, len_female_1000 - s2_female_1000 + 302, size=1) 
    phi_vals.append(eta(p1_boot) - eta(p2_boot))

lower_bound4 = np.percentile(phi_vals, lower_percentile)
upper_bound4 = np.percentile(phi_vals, upper_percentile)
print(f"Credible Interval for Phi: {lower_bound4, upper_bound4}")
print(f"Legnth of CI: {upper_bound4 - lower_bound4}")
print(f"Phi estimator: {np.mean(phi_vals)}")

Credible Interval for Phi: (0.3283202301727988, 0.6764165400892463)
Legnth of CI: 0.3480963099164475
Phi estimator: 0.495808799450889


### Q 2.5
__We observe that the majority of the confidence intervals we computed exhibit similarity, along with the estimated phi, with the exception of the final confidence interval
This last confidence interval is notably the shortest, indicating a higher level of confidence. This outcome is sensible given that it was derived from our historical data.__